In [1]:
import pandas as pd
import simplejson, requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from urllib.request import urlopen
from bs4 import BeautifulSoup

import re
import sys
import time

In [2]:
!pip install simplejson

In [2]:
parking = pd.read_csv(r'C:\Users\PC\Documents\광진구공모전\데이터\서울시 시영주차장 실시간 주차대수 정보.csv',encoding='euc-kr')

In [3]:
gwang_parking = parking[parking['주소'].str.contains('광진구')].reset_index(drop=True)
len(gwang_parking)

26

# KAKAO

In [19]:
url = "https://dapi.kakao.com/v2/local/search/keyword.json?"
apikey = '3b4911ca11f3c0507d0c677abf30fbd5'

In [28]:
gwang_parking.loc[1]

주차장코드                             172218
주차장 종류                                NW
주차장명                       자양4동 공영주차장(구)
주소                           광진구 자양동 5-5
전화번호                        02-2049-4595
총 주자면                                 83
평일 운영 시작시각(HHMM)                       0
평일 운영 종료시각(HHMM)                    2400
주말 운영 시작시각(HHMM)                       0
주말 운영 종료시각(HHMM)                    2400
공휴일 운영 시작시각(HHMM)                      0
공휴일 운영 종료시각(HHMM)                   2400
기본 주차 시간(분 단위)                       5.0
기본 주차 요금                           150.0
추가 단위 시간(분 단위)                       0.0
추가 단위 요금                             0.0
일 최대 요금                              0.0
현재 주차 차량수                             73
현재 주차 차량수 업데이트시간     2023-04-04 15:27:40
운영구분                                   1
노상 주차장 관리그룹번호                        NaN
유무료구분                                  Y
버스 기본 주차 요금                          0.0
버스 기본 주차 시간(분 단위)                    0.0
버스 추가 단위 요금     

In [17]:
query = gwang_parking['주소'][0]

In [20]:
r = requests.get( url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + apikey } )

In [22]:
r.json()

{'documents': [{'address_name': '서울 광진구 자양동 609-1',
   'category_group_code': 'PS3',
   'category_group_name': '어린이집,유치원',
   'category_name': '교육,학문 > 유아교육 > 어린이집',
   'distance': '',
   'id': '16505134',
   'phone': '02-447-8093',
   'place_name': '선린어린이집',
   'place_url': 'http://place.map.kakao.com/16505134',
   'road_address_name': '서울 광진구 뚝섬로43길 14',
   'x': '127.076917090624',
   'y': '37.5333758434749'}],
 'meta': {'is_end': True,
  'pageable_count': 1,
  'same_name': {'keyword': '', 'region': [], 'selected_region': '서울 광진구 자양동'},
  'total_count': 1}}

# Selenium

In [4]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
# 광진구 공영주차장 검색
kakao_map_search_url = f"https://map.kakao.com/?q={'광진구 공영주차장'}"
driver.get(kakao_map_search_url)
driver.implicitly_wait(2)
driver.find_element_by_xpath('//*[@id="info.searchHeader.message"]/div/div[2]/a').send_keys(Keys.ENTER)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

# 장소더보기 클릭
source = driver.page_source
driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
driver.implicitly_wait(2)
source = driver.page_source

names = []
addresses = []
scores = []
scores_link = []
scores_cnt = []
reviews_cnt = []

count = 1
print('start')
while True : 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    driver.implicitly_wait(5)
    driver.find_element_by_css_selector('#info\.search\.page\.no'+str(count)).send_keys(Keys.ENTER)
    print(count)

    time.sleep(4)
    source = driver.page_source
    driver.implicitly_wait(5)
    raw = BeautifulSoup(source, 'html.parser')
    datas = raw.select('#info\.search\.place\.list > li')

    for data in datas : 
        name = data.select_one('div.head_item.clickArea > strong > a.link_name').attrs['title']
        address = data.select_one('div.info_item > div.addr > p').attrs['title']
        score = data.select_one('div.rating.clickArea > span.score > em').attrs['title']
        score_link = data.select_one('div.rating.clickArea > span.score > a').attrs['href']
        score_cnt = data.select_one('div.rating.clickArea > span.score > a').get_text()
        review_cnt = data.select_one('div.rating.clickArea > a > em').get_text()

        names.append(name)
        addresses.append(address)
        scores.append(score)
        scores_link.append(score_link)
        scores_cnt.append(score_cnt)
        reviews_cnt.append(review_cnt)

        print(name)
    count += 1
    if count == 4 : 
        break


<ipython-input-4-0b15ee4654ed>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
<ipython-input-4-0b15ee4654ed>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="info.searchHeader.message"]/div/div[2]/a').send_keys(Keys.ENTER)
<ipython-input-4-0b15ee4654ed>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)


start


<ipython-input-4-0b15ee4654ed>:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#info\.search\.page\.no'+str(count)).send_keys(Keys.ENTER)


1
자양4동 공영주차장
화양동 공영주차장
능동공영주차장
구의1동 공영주차장
광진광장 공영주차장
구의3동 공영주차장
자양유수지 공영주차장
중곡3동공영주차장
구의2동공영주차장
송림기사식당길A 공영주차장
동서울노외공영 주차장
동서울호텔길A 공영주차장
송림식당길 노상공영주차장
중곡복개천길노상공영주차장
중곡동복개천 노상공영주차장
2
공영빌딩 주차장
자양4동 공영주차장 전기차충전소
자양2동유수지 공영주차장 전기차충전소
동서울호텔길B 공영주차장
구의3동 공영주차장 전기차충전소
능동공영주차장 전기차충전소
구의1동 공영주차장 개방화장실
중곡3동 마을공원 공영주차장 전기차충전소
구의3동 공영주차장 개방화장실
능동공영주차장 개방화장실
뚝섬한강공원 4주차장
구의2동주택가 공동주차장
뚝섬한강공원 2주차장
중곡시장 주차장
뚝섬한강공원 1주차장
3
뻥튀기공원 거주자주차장
용마산 남측 거주자주차장
중곡2동주차장
긴고랑길공동주차장
쏘카존 동서울 노상공영주차장(23~25번)
그린존 자양유수지 공영주차장
쏘카존 구의3동 공영주차장


In [5]:
print(len(names))
print(len(scores))
print(len(scores_link)) 
print(len(scores_cnt))
print(len(reviews_cnt))


37
37
37
37
37


In [6]:
df = pd.DataFrame(
    zip(names,addresses,scores,scores_cnt,scores_link,reviews_cnt),
    columns=['주차장명', '주소', '평점', '평점개수', '평점리뷰링크','리뷰수']
    )

## 위경도 가져오기

In [183]:
!pip install geopy

In [27]:
import requests, json
import numpy as np 
def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
    # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
    headers = {"Authorization": "KakaoAK 3b4911ca11f3c0507d0c677abf30fbd5"}
    api_json = json.loads(str(requests.get(url,headers=headers).text))
    try : 
        address = api_json['documents'][0]['address']
        crd = {"lat": str(address['y']), "lng": str(address['x'])}
        return crd
    except : 
        crd = {"lat": 'None', "lng": 'None'}
        return crd


latitude = []
longitude = []
for i in df['주소'] : 
    val = get_location(i)
    latitude.append(val['lat'])
    longitude.append(val['lng'])


In [29]:
df['위도'] = latitude
df['경도'] = longitude

In [32]:
df.head()

,주차장명,주소,평점,평점개수,평점리뷰링크,리뷰수,위도,경도
0,자양4동 공영주차장,서울 광진구 자양동 5-5 지상,3.5점,4건,https://place.map.kakao.com/11475801#comment,35,37.5386777385268,127.068185927821
1,화양동 공영주차장,서울 광진구 군자로 22,1.6점,8건,https://place.map.kakao.com/20589068#comment,24,37.5452972378302,127.071178761641
2,능동공영주차장,서울 광진구 천호대로112길 56,4.5점,2건,https://place.map.kakao.com/20589328#comment,20,37.5540417412775,127.08024326398
3,구의1동 공영주차장,서울 광진구 자양로18길 78,2.3점,8건,https://place.map.kakao.com/20513708#comment,4,37.5382711792426,127.087865052886
4,광진광장 공영주차장,서울 광진구 광나루로 389 광진광장 지하,0.0점,0건,https://place.map.kakao.com/11475808#comment,13,37.5476931913115,127.073213422355


In [101]:
df.to_csv('광진구_공영주차장_상세정보.csv',index=False,encoding='euc-kr')

# 별점 리뷰 크롤링

In [64]:
df['평점리뷰링크'][0]

'https://place.map.kakao.com/11475801#comment'

In [65]:
url = 'https://place.map.kakao.com/11475801#comment'
response = requests.get(url)
print(response)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

<Response [200]>


In [68]:
soup.select('#mArticle')

[]

In [66]:
soup.select('#mArticle > div.cont_evaluation > div.evaluation_review')

[]

# 서울시 주차정보안내시스템

In [140]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()

# 광진구 공영주차장 검색
seoul_parking = f"https://parking.seoul.go.kr/?lat=37.52462558&lon=126.89564732&jmCode=1577281"
driver.get(seoul_parking)
driver.implicitly_wait(2)
driver.find_element_by_css_selector('#inpBox2 > div.select_design.region_gu > a').send_keys(Keys.ENTER)
driver.find_element_by_css_selector('#inpBox2 > div.select_design.region_gu > ul > li:nth-child(7) > a').send_keys(Keys.ENTER)
driver.find_element_by_css_selector('#parking_search').send_keys(Keys.ENTER)
time.sleep(4)


source = driver.page_source
raw = BeautifulSoup(source, 'html.parser')
datas = raw.select('#pk_search_view > ul')



<ipython-input-140-467b354d5dac>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
<ipython-input-140-467b354d5dac>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#inpBox2 > div.select_design.region_gu > a').send_keys(Keys.ENTER)
<ipython-input-140-467b354d5dac>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#inpBox2 > div.select_design.region_gu > ul > li:nth-child(7) > a').send_keys(Keys.ENTER)
<ipython-input-140-467b354d5dac>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#parking_search').send_keys(Keys.ENTER)


In [141]:
one = []
two = []
three = []
four = []
five = []

for data in datas : 
    name = data.select_one('li > strong > a').get_text()
    print(name)
    one.append(name)
    parking_space = data.select_one('li > span:nth-child(2)').get_text()
    print(parking_space)
    two.append(parking_space)

    phone_num = data.select_one('li > span:nth-child(3)').get_text()
    three.append(phone_num)
    try:
        doro_address = data.select_one('li > span:nth-child(4)').get_text()
        four.append(doro_address)
    except : 
        four.append('-')
    try : 
        zibon_address = data.select_one('li > span:nth-child(5)').get_text()
        five.append(zibon_address)
    except : 
        five.append('-')

AJ파크 커먼그라운드 주차장			
전체주차면 : 27면 기본정보
강변 테크노마트			
전체주차면 : 2184면 기본정보
건국대학교 동문회관 주차장			
전체주차면 : 51면 기본정보
건국대학교병원 주차장			
전체주차면 : 659면 | 주차가능면 : 353면
건국대학교사범대학부속중학교 주차장			
전체주차면 : 100면 | 주차가능면 : 10면
경동 주차장(민영)			
전체주차면 : 140면 기본정보
공유) 광장동 IoT 공유주차장(구)			
전체주차면 : 39면 기본정보
공유) 능동 IoT 공유주차장(구)			
전체주차면 : 11면 기본정보
공유) 능동로24길 IoT 공유주차장(구)			
전체주차면 : 17면 기본정보
광성빌딩 주차장			
전체주차면 : 43면 기본정보
광진광장 공영주차장(구)			
전체주차면 : 33면 기본정보
광진구민체육센타 및 광진청소년수련관			
전체주차면 : 88면 기본정보
광진구청			
전체주차면 : 60면 기본정보
광진문화예술회관 및 구의회청사 주차장			
전체주차면 : 140면 기본정보
광진유료주차장(민영)			
전체주차면 : 35면 기본정보
광진초등학교			
전체주차면 : 53면 기본정보
구의1동 공영주차장(구)			
전체주차면 : 172면 기본정보
구의2동 주택가 공영주차장(구)			
전체주차면 : 90면 기본정보
구의3동 공영주차장(구)			
전체주차면 : 159면 기본정보
구의웰츠타워 주차장			
전체주차면 : 274면 기본정보
구의주차장(민영)			
전체주차면 : 34면 기본정보
국립정신건강센터			
전체주차면 : 481면 기본정보
그랜드파크 오피스텔			
전체주차면 : 74면 기본정보
글로벌시티오피스텔 주차장			
전체주차면 : 45면 기본정보
능동공영주차장(구)			
전체주차면 : 77면 기본정보
대승주차장011-259-7583(민영)			
전체주차면 : 60면 기본정보
도광빌딩 주차장			
전체주차면 : 45면 기본정보
동서울 노외주차장(구)			
전체주차면 : 25면 기본정보
동서울 호텔길(B) 공영주차장(구)	

In [142]:
df2 = pd.DataFrame(
    zip(one,two,three,four,five),
    columns=['주차장명', '주차공간', '번호', '도로주소', '지번주소']
    )

In [143]:
df2['주차장명'] = df2['주차장명'].str.rstrip()

In [144]:
df2.to_csv('서울시_주차정보_시스템2.csv',index=False,encoding='euc-kr')

# 네이버 크롤링

In [173]:
kakao = pd.read_csv(r'C:\Users\PC\Documents\광진구공모전\광진구_공영주차장_상세정보.csv',encoding='euc-kr')
kakao.head()

,주차장명,주소,평점,평점개수,평점리뷰링크,리뷰수,위도,경도
0,자양4동 공영주차장,서울 광진구 자양동 5-5 지상,3.5점,4건,https://place.map.kakao.com/11475801#comment,35,37.5386777385268,127.068185927821
1,화양동 공영주차장,서울 광진구 군자로 22,1.6점,8건,https://place.map.kakao.com/20589068#comment,24,37.5452972378302,127.071178761641
2,능동공영주차장,서울 광진구 천호대로112길 56,4.5점,2건,https://place.map.kakao.com/20589328#comment,20,37.5540417412775,127.08024326398
3,구의1동 공영주차장,서울 광진구 자양로18길 78,2.3점,8건,https://place.map.kakao.com/20513708#comment,4,37.5382711792426,127.087865052886
4,광진광장 공영주차장,서울 광진구 광나루로 389 광진광장 지하,0.0점,0건,https://place.map.kakao.com/11475808#comment,13,37.5476931913115,127.073213422355


In [175]:
parking_list = kakao['주차장명']

In [214]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
# 광진구 공영주차장

driver.get("https://map.naver.com/v5/")
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

parks_names = []
names = []
stars = []
review_urls = []
review_cnts = []
blog_review_cnts = []

for park in parking_list : 
    element.send_keys(park)
    element.send_keys(Keys.ENTER)
    time.sleep(3)
    
    # WebDriverWait(driver,1.5).until(
    #     EC.presence_of_element_located((By.ID, 'searchIframe'))
    #     )
    driver.switch_to.frame('searchIframe')
    time.sleep(3)
    try : 
        driver.find_element_by_css_selector('#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div > a:nth-child(1)').send_keys(Keys.ENTER)
        time.sleep(5)
        # WebDriverWait(driver,1.5).until(
        #     EC.presence_of_element_located((By.ID, 'entryIframe'))
        #     )
        driver.switch_to.default_content()
        driver.switch_to.frame('entryIframe')

        source = driver.page_source
        raw = BeautifulSoup(source, 'html.parser')
        datas = raw.select('#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0')
        for data in datas : 
                name = data.select_one('#_title > span.Fc1rA').get_text()
                
                real_name = name.replace(' ','')
                real_park = park.replace(' ','')
                print('park :',real_park)
                print('name :',real_name)
                names.append(real_name)

                try : 
                    star = data.select_one('div.dAsGb > span.PXMot.LXIwF > em').get_text()
                    stars.append(star)
                except : 
                    stars.append('-')
                try : 
                    review_url = 'https://pcmap.place.naver.com'+data.select_one('div.dAsGb > span:nth-child(2) > a').attrs['href']
                    review_urls.append(review_url)
                except : 
                    review_urls.append('-')
                try : 
                    review_cnt = data.select_one('div.dAsGb > span:nth-child(2) > a > em').get_text()
                    review_cnts.append(review_cnt)
                except : 
                    review_cnts.append('-')
                try :
                    blog_review_cnt = data.select_one('div.dAsGb > span:nth-child(3) > a > em').get_text()
                    blog_review_cnts.append(blog_review_cnt)
                except : 
                    blog_review_cnts.append('-')


    except : 
        names.append('-')
        stars.append('-')
        review_urls.append('-')
        review_cnts.append('-')
        blog_review_cnts.append('-')
    
    parks_names.append(park)
                    
    driver.switch_to.default_content()
    driver.find_element(By.CLASS_NAME,"button_clear.ng-star-inserted").send_keys(Keys.ENTER)
    time.sleep(3)

<ipython-input-214-14d1554d1e09>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
<ipython-input-214-14d1554d1e09>:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div > a:nth-child(1)').send_keys(Keys.ENTER)


park : 자양4동공영주차장
name : 자양4동공영주차장
park : 화양동공영주차장
name : 화양동공영주차장
park : 능동공영주차장
name : 능동공영주차장
park : 구의1동공영주차장
name : 구의1동공영주차장
park : 광진광장공영주차장
name : 광진광장공영주차장
park : 구의3동공영주차장
name : 구의3동공영주차장
park : 자양유수지공영주차장
name : 자양유수지노상공영주차장
park : 중곡3동공영주차장
name : 중곡3동공영주차장
park : 구의2동공영주차장
name : 구의1동공영주차장
park : 동서울호텔길A공영주차장
name : 동서울호텔길A노상공영주차장
park : 송림식당길노상공영주차장
name : 송림식당길노상공영주차장
park : 중곡복개천길노상공영주차장
name : 중곡복개천길공영노상주차장
park : 중곡동복개천노상공영주차장
name : 중곡복개천노상공영주차장
park : 공영빌딩주차장
name : 유료민영주차장
park : 자양4동공영주차장전기차충전소
name : 자양4동공영주차장전기차충전소
park : 동서울호텔길B공영주차장
name : 동서울호텔길B노상공영주차장
park : 구의3동공영주차장전기차충전소
name : 구의3동공영주차장전기차충전소
park : 능동공영주차장전기차충전소
name : 능동공영주차장전기차충전소
park : 중곡3동마을공원공영주차장전기차충전소
name : 중곡3동마을공원공영주차장전기차충전소
park : 뚝섬한강공원4주차장
name : 한강공원잠실4주차장
park : 구의2동주택가공동주차장
name : 구의2동주택가공동주차장
park : 뚝섬한강공원2주차장
name : 뚝섬한강공원2주차장
park : 중곡시장주차장
name : 경동민영주차장
park : 뚝섬한강공원1주차장
name : 잠실한강공원잠실제1주차장
park : 중곡2동주차장
name : 중곡3동공영주차장
park : 쏘카존구의3동공영주차장
name : 쏘카구의3동공영주차장


In [215]:
naver = pd.DataFrame(
    zip(parks_names,names,stars,review_urls,review_cnts,blog_review_cnts),
    columns=['카카오기준_주차장명','주차장명', '평점', '리뷰url', '리뷰수', '블로그리뷰수']
    )

In [217]:
naver.to_csv('네이버_광진구_공영주차장정보.csv',index=False,encoding='euc-kr')

# 리뷰데이터 수집

## 네이버에서 크롤링한 틀린 정보 일부 엑셀에서 수정 후 다시 재로드

In [279]:
naver2 = pd.read_csv(r'C:\Users\PC\Documents\광진구공모전\네이버_광진구_공영주차장정보.csv',encoding='euc-kr')

In [306]:
print(naver2.shape)
naver2.head()

(37, 6)


,카카오기준_주차장명,주차장명,평점,리뷰url,리뷰수,블로그리뷰수
0,자양4동 공영주차장,자양4동공영주차장,4.46,https://pcmap.place.naver.com/place/21185991/r...,586,74
1,화양동 공영주차장,화양동공영주차장,4.23,https://pcmap.place.naver.com/place/18832374/r...,503,47
2,능동공영주차장,능동공영주차장,4.34,https://pcmap.place.naver.com/place/18810610/r...,567,30
3,구의1동 공영주차장,구의1동공영주차장,4.45,https://pcmap.place.naver.com/place/35992895/r...,98,18
4,광진광장 공영주차장,광진광장공영주차장,4.34,https://pcmap.place.naver.com/place/21185766/r...,301,24


In [307]:
links = naver2[naver2['리뷰url']!='-']
print(links.shape)
links

(14, 6)


,카카오기준_주차장명,주차장명,평점,리뷰url,리뷰수,블로그리뷰수
0,자양4동 공영주차장,자양4동공영주차장,4.46,https://pcmap.place.naver.com/place/21185991/r...,586,74
1,화양동 공영주차장,화양동공영주차장,4.23,https://pcmap.place.naver.com/place/18832374/r...,503,47
2,능동공영주차장,능동공영주차장,4.34,https://pcmap.place.naver.com/place/18810610/r...,567,30
3,구의1동 공영주차장,구의1동공영주차장,4.45,https://pcmap.place.naver.com/place/35992895/r...,98,18
4,광진광장 공영주차장,광진광장공영주차장,4.34,https://pcmap.place.naver.com/place/21185766/r...,301,24
5,구의3동 공영주차장,구의3동공영주차장,4.42,https://pcmap.place.naver.com/place/18816578/r...,90,10
6,자양유수지 공영주차장,자양유수지노상공영주차장,-,https://pcmap.place.naver.com/place/1818015545...,4,-
7,중곡3동공영주차장,중곡3동공영주차장,3.38,https://pcmap.place.naver.com/place/1078698479...,30,1
8,구의2동공영주차장,구의2동주택가공동주차장,4.41,https://pcmap.place.naver.com/place/226115480/...,223,24
11,동서울호텔길A 공영주차장,동서울호텔길A노상공영주차장,-,https://pcmap.place.naver.com/place/624841151/...,1,-


In [292]:
tmp = pd.DataFrame()

# 광진구 공영주차장
for park, link in zip(links['카카오기준_주차장명'],links['리뷰url']): 
    print(park)
    driver = webdriver.Chrome('chromedriver.exe')
    driver.maximize_window()
    driver.get(link)
    count = 0
    while count < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2)
        try : 
            # 장소더보기 클릭
            driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.lcndr > div.lfH3O > a').send_keys(Keys.ENTER)
            count += 1
        except : 
            break
        time.sleep(2)

    
    source = driver.page_source
    raw = BeautifulSoup(source, 'html.parser')
    datas = raw.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.lcndr > div.place_section_content > ul > li')

    parking_review = []
    parking = []
    for data in datas : 
        try : 
            review = data.select_one('.zPfVt').get_text()
            parking_review.append(review)
            parking.append(park)
        except : 
            pass
    driver.close()

    temp = pd.DataFrame(zip(parking,parking_review),columns=['주차장명','리뷰'])
    tmp = pd.concat([tmp,temp],axis=0)


자양4동 공영주차장


<ipython-input-292-8dfb26c7bbdb>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
<ipython-input-292-8dfb26c7bbdb>:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.lcndr > div.lfH3O > a').send_keys(Keys.ENTER)


화양동 공영주차장
능동공영주차장
구의1동 공영주차장
광진광장 공영주차장
구의3동 공영주차장
자양유수지 공영주차장
중곡3동공영주차장
구의2동공영주차장
동서울호텔길A 공영주차장
동서울호텔길B 공영주차장
뚝섬한강공원 4주차장
구의2동주택가 공동주차장
뚝섬한강공원 2주차장


In [315]:
ddf = tmp

In [319]:
links

,카카오기준_주차장명,주차장명,평점,리뷰url,리뷰수,블로그리뷰수
0,자양4동 공영주차장,자양4동공영주차장,4.46,https://pcmap.place.naver.com/place/21185991/r...,586,74
1,화양동 공영주차장,화양동공영주차장,4.23,https://pcmap.place.naver.com/place/18832374/r...,503,47
2,능동공영주차장,능동공영주차장,4.34,https://pcmap.place.naver.com/place/18810610/r...,567,30
3,구의1동 공영주차장,구의1동공영주차장,4.45,https://pcmap.place.naver.com/place/35992895/r...,98,18
4,광진광장 공영주차장,광진광장공영주차장,4.34,https://pcmap.place.naver.com/place/21185766/r...,301,24
5,구의3동 공영주차장,구의3동공영주차장,4.42,https://pcmap.place.naver.com/place/18816578/r...,90,10
6,자양유수지 공영주차장,자양유수지노상공영주차장,-,https://pcmap.place.naver.com/place/1818015545...,4,-
7,중곡3동공영주차장,중곡3동공영주차장,3.38,https://pcmap.place.naver.com/place/1078698479...,30,1
8,구의2동공영주차장,구의2동주택가공동주차장,4.41,https://pcmap.place.naver.com/place/226115480/...,223,24
11,동서울호텔길A 공영주차장,동서울호텔길A노상공영주차장,-,https://pcmap.place.naver.com/place/624841151/...,1,-


In [318]:
tmp['주차장명'].value_counts()

구의3동 공영주차장       40
구의2동공영주차장        40
구의2동주택가 공동주차장    40
자양4동 공영주차장       39
화양동 공영주차장        39
능동공영주차장          39
구의1동 공영주차장       39
광진광장 공영주차장       39
뚝섬한강공원 4주차장      34
중곡3동공영주차장        26
자양유수지 공영주차장       5
동서울호텔길A 공영주차장     2
Name: 주차장명, dtype: int64

In [316]:
ddf

,주차장명,리뷰
0,자양4동 공영주차장,굿굿
1,자양4동 공영주차장,주차요금 저렴하고 좋아요!!
2,자양4동 공영주차장,건대입구역 근처에 음식점 가려면 여기 갠춘
3,자양4동 공영주차장,공영주차장이라 주차요금이 저렴해요!👍👍
4,자양4동 공영주차장,경차할인되고 좋아요
...,...,...
35,구의2동주택가 공동주차장,굿
36,구의2동주택가 공동주차장,친절해요
37,구의2동주택가 공동주차장,집근처라 지인방문시 잘 이용하고 있습니다
38,구의2동주택가 공동주차장,굿


In [320]:
drop_links = links[(links['카카오기준_주차장명']=='동서울호텔길B 공영주차장')|(links['카카오기준_주차장명']=='뚝섬한강공원 2주차장')]

In [321]:
tmp2 = pd.DataFrame()
for park, link in zip(drop_links['카카오기준_주차장명'],drop_links['리뷰url']) : 
    driver = webdriver.Chrome('chromedriver.exe')
    driver.maximize_window()
    driver.get(link)
    count = 0
    while count < 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2)
        try : 
            driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').send_keys(Keys.ENTER)
            count += 1
        except : 
            break
        time.sleep(2)
    source = driver.page_source
    raw = BeautifulSoup(source, 'html.parser')

    datas = raw.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.place_section_content > ul > li')
    parking_review = []
    parking = []
    for data in datas : 
        try : 
            review = data.select_one('.zPfVt').get_text()
            parking_review.append(review)
            parking.append(park)
        except : 
                pass
    temp2 = pd.DataFrame(zip(parking,parking_review),columns=['주차장명','리뷰'])
    tmp2 = pd.concat([tmp2,temp2],axis=0)
    driver.close()

<ipython-input-321-31575c528d20>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
<ipython-input-321-31575c528d20>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section.lcndr > div.lfH3O > a').send_keys(Keys.ENTER)


In [326]:
final_review = pd.concat([tmp,tmp2],axis=0)
final_review = final_review.reset_index(drop=True)

In [327]:
final_review

,주차장명,리뷰
0,자양4동 공영주차장,굿굿
1,자양4동 공영주차장,주차요금 저렴하고 좋아요!!
2,자양4동 공영주차장,건대입구역 근처에 음식점 가려면 여기 갠춘
3,자양4동 공영주차장,공영주차장이라 주차요금이 저렴해요!👍👍
4,자양4동 공영주차장,경차할인되고 좋아요
...,...,...
420,동서울호텔길B 공영주차장,굿
421,동서울호텔길B 공영주차장,주차라인은 좁지만 있어서 좋음
422,뚝섬한강공원 2주차장,한쪽 방향에서만 입차가 가능한데 종전과 방향이 달라서 한참 돌아 대기했네요
423,뚝섬한강공원 2주차장,최고좋아요 또방문예정!


In [328]:
final_review['주차장명'].value_counts()

구의3동 공영주차장       40
구의2동공영주차장        40
구의2동주택가 공동주차장    40
동서울호텔길B 공영주차장    40
자양4동 공영주차장       39
화양동 공영주차장        39
능동공영주차장          39
구의1동 공영주차장       39
광진광장 공영주차장       39
뚝섬한강공원 4주차장      34
중곡3동공영주차장        26
자양유수지 공영주차장       5
뚝섬한강공원 2주차장       3
동서울호텔길A 공영주차장     2
Name: 주차장명, dtype: int64

In [336]:
final_review.to_csv('네이버_광진구_공영주차장_리뷰.csv',index=False,encoding='utf-8')

In [341]:
kakao.rename(columns={'주차장명':'카카오기준_주차장명'},inplace=True)

In [345]:
kakao_naver = pd.merge(kakao,naver2,on='카카오기준_주차장명')[['카카오기준_주차장명','주소','평점_x','평점_y','평점개수','리뷰수_x','리뷰수_y','블로그리뷰수','위도','경도']]
kakao_naver.rename(columns={
    '평점_x' : '카카오평점',
    '평점_y' : '네이버평점',
    '평점개수' :'카카오평점수',
    '리뷰수_x' : '카카오리뷰수',
    '리뷰수_y' : '네이버리뷰수',
    '블로그리뷰수' : '네이버블로그리뷰수'
},inplace=True)

In [347]:
kakao_naver.to_csv('카카오_네이버_광진구_공영주차장정보.csv',index=False, encoding='euc-kr')